In [2]:
import os
from dotenv import load_dotenv , find_dotenv

_ =load_dotenv(find_dotenv())


from langchain_groq import ChatGroq

chatm = ChatGroq(
    model="llama3-70b-8192"
)

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

chain = prompt | chatm | output_parser
chain.invoke({"soccer_player":"Lionel Messi"})

"Here's one:\n\nLionel Messi has a growth hormone deficiency, which was diagnosed when he was just 11 years old. As a result, he had to undergo hormone therapy to help him grow. In fact, his family had to make huge sacrifices to pay for his treatment, which cost around $1,500 per month. This condition, combined with his small stature, made many coaches and scouts doubt his ability to succeed as a professional footballer. However, Messi's determination, hard work, and natural talent ultimately prevailed, and he went on to become one of the greatest footballers of all time!"

In [4]:
#Builtin Functions
#Use of .bind() to add arguments to a runnable in a LCEL chain

chain = prompt | chatm.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player":"Ronaldo"})

"Here's one:\n\n**Cristiano "

Combining Chains

In [7]:
#simple chain
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")
chain = prompt | chatm | output_parser

#combining chain

his_prompt = ChatPromptTemplate.from_template("was {soccer_player} player good or bad ?")

composed_chain = {"soccer_player":chain} | his_prompt | chatm | output_parser
composed_chain.invoke({"soccer_player":"Lionel Messi"})


"What an amazing story! I'd definitely say that Lionel Messi is an inspiring and talented player! Despite facing a significant medical challenge, he overcame it with determination and hard work, and went on to achieve incredible success in his career. His rags-to-riches story is truly motivational, and it's wonderful to see how FC Barcelona's support helped him receive the necessary treatment. It just goes to show that with perseverance and the right support, anyone can achieve their dreams!"

In [10]:
#another example for combining chain

from operator import itemgetter


prompt1 = ChatPromptTemplate.from_template("What is the country{politician} is from?")
prompt2 = ChatPromptTemplate.from_template("What continent is the country {country} in? respond in {language}")

chain1 = prompt1 | chatm | output_parser

chain2 = ({"country":chain1,"language":itemgetter("language")}) | prompt2 | chatm | output_parser

chain2.invoke({"politician":"Miterrand","language":"French"})

"Une question facile !\n\nLa réponse est : L'Europe. François Mitterrand a été président de la France de 1981 à 1995, et la France est située sur le continent européen."

LCEL Chain at work in a typical RAG app

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.doument_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits,embeddings=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context":retriever | format_docs,"question":RunnablePassthrough()}
    | prompt
    | chatm
    | output_parser
)
rag_chain.invoke("What is task decomposition?")